Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [98]:
import numpy as np
from copy import deepcopy
from icecream import ic

In [99]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [100]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) ## valori degli oggetti
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) ## pesi degli oggetti (n.b ogni peso ha più dimensioni e i constraint devono essere rispettati in ogni dimensione)
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) ## tante quante le dimensioni 

In [101]:
CONSTRAINTS
# ogni zaino è una riga e ogni colonna è il constraint sulla dimensione

array([[ 14, 366],
       [182, 345]], dtype=int32)

In [5]:
def flip_item(sol):
    new_sol = np.copy(sol)
    k = np.random.randint(NUM_KNAPSACKS)
    i = np.random.randint(NUM_ITEMS)
    new_sol[k, i] = not new_sol[k, i]
    return new_sol

def move_item(sol):
    new_sol = np.copy(sol)
    i = np.random.randint(NUM_ITEMS)
    from_k = np.argmax(sol[:, i]) if np.any(sol[:, i]) else None
    to_k = np.random.randint(NUM_KNAPSACKS)

    if from_k is not None:
        new_sol[from_k, i] = False
    new_sol[to_k, i] = True
    return new_sol

def swap_items(sol):
    new_sol = np.copy(sol)
    k1, k2 = np.random.choice(NUM_KNAPSACKS, 2, replace=False)
    i1, i2 = np.random.randint(NUM_ITEMS, size=2)
    new_sol[k1, i1], new_sol[k2, i2] = new_sol[k2, i2], new_sol[k1, i1]
    return new_sol


In [6]:
def tweak(solution):
    new_solution = np.copy(solution)

    # scegli casualmente un'operazione di modifica
    move_type = np.random.choice(["flip", "move", "swap"], p=[0.5, 0.3, 0.2])
    
    if move_type == "flip":
        new_solution = flip_item(new_solution)
    elif move_type == "move":
        new_solution = move_item(new_solution)
    elif move_type == "swap":
        new_solution = swap_items(new_solution)
    
    return new_solution


In [7]:
def enforce_unique_assignment(sol):
    # se un item è in più knapsack, lascialo in uno solo
    for i in range(NUM_ITEMS):
        assigned = np.where(sol[:, i])[0]
        if len(assigned) > 1:
            keep = np.random.choice(assigned)
            sol[:, i] = False
            sol[keep, i] = True
    return sol

In [92]:
# fitness function: higher is better
def fitness_cost(solution: list[set] ) -> (float, float):
    total_value = 0
    penalty = 0.0

    for b in range(NUM_KNAPSACKS): 
        knapsack_weight = np.zeros(NUM_DIMENSIONS)
        knapsack_value = 0
        for i in range(NUM_ITEMS):
            if solution[b][i]:
                knapsack_value += VALUES[i]
                knapsack_weight += WEIGHTS[i]

        total_value += knapsack_value

        # penalità proporzionale all'eccesso
        overweight = knapsack_weight - CONSTRAINTS[b]
        penalty += np.sum(np.maximum(0, overweight))

    
    # il parametro di bilanciamento dinamico lo aumento, inizialmente è più piccolo per permettere 
    # l'esplorazione poi lo aumento per penalizzare/escludere soluuzioni non valide
    alpha =  10
    # fitness = valore totale - penalità pesata
    fitness = total_value - alpha * penalty

    return (fitness, penalty)


In [93]:

current_solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
for item in range(NUM_ITEMS):
    if np.random.random() < 0.5:  # probabilità di assegnare l'oggetto
        knapsack = np.random.randint(NUM_KNAPSACKS)
        current_solution[knapsack, item] = True
alpha_start = 1
alpha_end = 50
MAX_STEPS = 500

current_fitness, current_penality = fitness_cost(current_solution)




for step in range(MAX_STEPS):

   # ic(current_solution, current_cost)
    new_solution= tweak(current_solution)
    new_solution= enforce_unique_assignment(new_solution)
    new_fitness, new_penality = fitness_cost(new_solution)
    if new_fitness> current_fitness: #cost in realtà è una fitness function
        current_solution= new_solution
        current_fitness= new_fitness
        current_penality= new_penality

current_solution
current_fitness
        

np.float64(-2226.0)

In [94]:
solution= current_solution
solution

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, Fals

In [95]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.True_

In [96]:
# Check if the solution is valid
#all_knapsacks = np.any(solution, axis=0)
#np.all(WEIGHTS[all_knapsacks].sum(axis=0) < CONSTRAINTS)

valid_per_bag = []
for b in range(NUM_KNAPSACKS):
    # seleziona gli item presenti nello zaino b
    items_in_b = np.where(solution[b])[0]            # indici degli item presenti
    total_weight = WEIGHTS[items_in_b].sum(axis=0)   # somma per dimensione
    valid_per_bag.append(np.all(total_weight <= CONSTRAINTS[b]))

overall_valid = all(valid_per_bag)
overall_valid

False

## TEST PROBLEMS

In [40]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS

array([[535, 102],
       [ 73,  26],
       [339, 135]], dtype=int32)

In [74]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS

array([[1197, 8295, 6548, 8012, 8031, 5119, 5976, 2500,   88, 5008],
       [9859, 1380, 9587,  343, 9358, 4101, 6168, 9636, 7347, 6733],
       [1713, 1344, 2968, 9165, 9195, 8716, 2057,  401, 1890, 9079],
       [9103, 4219, 4055, 5591, 6670, 5296, 9329, 5027, 3293,  914],
       [2695, 7469, 9167, 7644, 9039, 3425, 8202, 3205, 5590, 7943],
       [7727, 8297, 7909, 6873, 5913, 6778, 7140, 5666, 6345, 8384],
       [9081, 6853, 9189, 1907, 6371, 3232, 8271, 1750, 4700, 8173],
       [4853, 2877, 1493, 9340, 1643, 4278,  559, 9480, 1889, 7633],
       [1017, 5652, 2816, 4093, 6878, 8991,  620, 7631, 1979, 6797],
       [7395, 6978, 5607, 4885, 8609, 4449, 5062, 1005, 5847, 1964]],
      dtype=int32)

In [ ]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 



OLD SOLUTION 

In [ ]:
#### OLD SOLUTION# --- IGNORE ---

def tweak_non_valida(knapsacs:  list[set] ) -> list[set]:
    ## conviene togliere un oggetto da uno zaino che è in sovrappeso
    new_bags = deepcopy(knapsacs)
    # trova uno zaino in sovrappeso
    overweight_bags = []
    for b in range(NUM_KNAPSACKS):
        total_weight = np.zeros(NUM_DIMENSIONS)
        for i in range(NUM_ITEMS):
            if new_bags[b][i]:
                total_weight += WEIGHTS[i]
        if np.any(total_weight > CONSTRAINTS[b]):
            overweight_bags.append(b)
            

    for b in overweight_bags:
        items_to_remove= []
        for i in range(NUM_ITEMS):
            if new_bags[b][i]:
                items_to_remove.append(i)
        
        if len(items_to_remove) > 0:
            item_to_remove = np.random.choice(items_to_remove)
            new_bags[b][item_to_remove] = False
            break  # tweak only one bag
    return new_bags

def tweak_valida(knapsack: list[set]) -> list[set]:
    ## conviene aggiungere un oggetto a uno zaino che non è in sovrappeso
    new_bags = deepcopy(knapsack)
   
    b= np.random.randint(NUM_KNAPSACKS)
    # calcolo il peso attuale dello zaino
        
    knapsack_weight = np.zeros(NUM_DIMENSIONS)
    for i in range(NUM_ITEMS):
        if new_bags[b][i]:
            knapsack_weight += WEIGHTS[i]
        
    ## prendo un oggetto a caso se non è presente in nessuno zaino lo aggiungo a quello più vuoto al contrario lo scambio
    item_to_add = np.random.choice([i for i in range(NUM_ITEMS) ])
        
    if item_to_add in new_bags[b]:
    # lo tolgo e lo metto in un altro zaino
        new_bags[b][item_to_add]=False
        other_bags = [i for i in range(NUM_KNAPSACKS) if i != b]
        other_bag = np.random.choice(other_bags)
        new_bags[other_bag][item_to_add] = True
    else:
            ## lo aggiungo e lo tolgo da un altro zaino se presente
        new_bags[b][item_to_add]=True
        new_knapsack_weight = knapsack_weight+ WEIGHTS[item_to_add]
        if np.any(new_knapsack_weight > CONSTRAINTS[b]):
            # non posso aggiungerlo
            new_bags[b][item_to_add]=False            
    return new_bags

#OLD SOLUTION
## il costo di una soluzione è dato dalla somma dei valori degli oggetti meno la somma dei sovrappesi
# se la soluzione è valida il costo è negativo
def cost(solution: list[set]) -> (float, float, bool):
    total_value = 0
    total_overweight = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))

    for b in range(NUM_KNAPSACKS): 
        knapsack_weight = np.zeros(NUM_DIMENSIONS)
        knapsack_value = 0
        for i in range(NUM_ITEMS):
            if solution[b][i]:
                knapsack_value += VALUES[i]
                knapsack_weight += WEIGHTS[i]
        total_value += knapsack_value
        total_overweight[b] = knapsack_weight - CONSTRAINTS[b]

    # validità: nessuna componente > 0
    valid = not np.any(total_overweight > 0)

    # penalità: somma delle eccedenze
    penalty = np.sum(np.maximum(0, total_overweight))
    # fitness: valore - penalità pesata
    alpha = 10
    fitness = total_value - alpha * penalty

    return fitness, penalty, valid



current_solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
for item in range(NUM_ITEMS):
    if np.random.random() < 0.5:  # probabilità di assegnare l'oggetto
        knapsack = np.random.randint(NUM_KNAPSACKS)
        current_solution[knapsack, item] = True

(current_fitness, current_cost, current_valid) = cost(current_solution)


MAX_STEPS = 500


for steps in range(MAX_STEPS):

    ic(steps, current_cost, current_fitness, current_valid)
    if current_valid:
        ## soluzione valida
        
        new_solution = tweak_valida(current_solution)
        (new_fitness, new_cost, new_valid) = cost(new_solution)
        if new_valid:
            if new_fitness> current_fitness: 
                ## la nuova soluzione è valida e il costo si avvicina a zero sempre di più (costo ideale)
                current_cost= new_cost
                current_fitness= new_fitness
                current_solution = new_solution
                current_valid= new_valid
                continue

    else: 
        ## soluzione non valida
        new_solution = tweak_non_valida(current_solution)
        (new_fitness, new_cost, new_valid) = cost(new_solution)

        if not new_valid: 
            
        ## la nuova soluzione non è valida anche essa 
            if new_cost < current_cost: # 0<= new < current
                # la nuova soluzione si avvicina a essere valida 
                current_cost = new_cost
                current_solution = new_solution
                current_fitness= new_fitness
                current_valid= new_valid
                continue
            else:
                continue
        else:
            ## la nuova soluzione è valida 
            current_cost = new_cost
            current_solution = new_solution
            current_fitness= new_fitness
            current_valid= new_valid
            continue

    

print(current_cost, current_fitness, current_valid)



ic| steps: 0
    current_cost: np.float64(68.0)
    current_fitness: np.float64(-428.0)
    current_valid: False
ic| steps: 1
    current_cost: np.float64(10.0)
    current_fitness: np.float64(83.0)
    current_valid: False


0.0 176.0 True


In [103]:
solution=current_solution
solution

array([[False, False, False, False, False, False, False, False, False,
        False],
       [ True, False, False,  True, False, False, False,  True, False,
        False]])

In [104]:
# Check if the solution is valid
valid_per_bag = []
for b in range(NUM_KNAPSACKS):
    # seleziona gli item presenti nello zaino b
    items_in_b = np.where(solution[b])[0]            # indici degli item presenti
    total_weight = WEIGHTS[items_in_b].sum(axis=0)   # somma per dimensione
    valid_per_bag.append(np.all(total_weight <= CONSTRAINTS[b]))

overall_valid = all(valid_per_bag)
overall_valid

True

In [105]:
def solution_evaluation(solution):
    total_value = 0
    percentage_weight_over_constraint = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))

    for b in range(NUM_KNAPSACKS): 
        knapsack_weight = np.zeros(NUM_DIMENSIONS)
        knapsack_value = 0
        for i in range(NUM_ITEMS):
            if solution[b][i]:
                knapsack_value += VALUES[i]
                knapsack_weight += WEIGHTS[i]
        total_value += knapsack_value
        percentage_weight_over_constraint[b] = knapsack_weight*100/CONSTRAINTS[b]


    return percentage_weight_over_constraint, total_value

solution_evaluation(solution)

(array([[ 0.        ,  0.        ],
        [98.9010989 , 25.50724638]]),
 np.int32(156))